## 3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to features
3. reconstruct paleolocations for features


In [27]:
import sys
sys.path.insert(1, './pygplates_rev28_python38_MacOS64/')
import pygplates

import pandas as pd

import os

# define plate model

# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection('PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')

# The partition_into_plates function requires a rotation model, since sometimes this would be
# necessary even at present day (for example to resolve topological polygons)
rotation_model=pygplates.RotationModel('PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

In [28]:
expts          = ['teXpz', 'teXpy', 'teXpx', 'teXpw', 'teXpv', 'teXpu', 'teXpt', 'teXps', 'teXpr', 'teXpq', 'teXpp', 'teXpn']
labels         = ['385Ma', '380Ma', '375Ma', '370Ma', '366Ma', '359Ma', '354Ma', '349Ma', '344Ma', '339Ma', '333Ma', '327Ma']
ages           = ['385', '380', '375', '370', '366', '359', '354', '349', '344', '339', '333', '327']


In [30]:
# reconstruct list of points

# load point coordinates
df_sites = pd.read_csv('./NA-sites.csv',sep=',')

for expCount, exp in enumerate(expts):
    
    # Create target directory & all intermediate directories if don't exists
    dirName = './reconstructed-shapes/'+ labels[expCount]
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists") 
    
    #### step 1: put the points into a feature collection, using Lat,Lon coordinates from dataframe
    point_features = []
    for index,row in df_sites.iterrows():
        point = pygplates.PointOnSphere(float(row.LAT),float(row.LON))
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_features.append(point_feature)

    ### step 2: assign plate ids to features
    # To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
    # then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
    # menu option in GPlates GUI) 
    partitioned_point_features = pygplates.partition_into_plates(static_polygons, rotation_model, point_features) 

    ### step 3: reconstruct paleolocations for features
    # Two possible ways:
    
    # 1. save shape files to disk for later use (e.g. load sahpefiles into python script for direct plotting)
    pygplates.reconstruct(partitioned_point_features, rotation_model, dirName +'/NA-sites.shp', float(ages[expCount]))
    
    # 2. output paleolocations directly
    reconstructed_feature_geometries = []
    pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, float(ages[expCount]))    
    for siteCount, reconstructed_feature_geometry in enumerate(reconstructed_feature_geometries):
        print('Paleolocation (LAT, LON) for ' + df_sites.name[siteCount] + 'at ' + labels[expCount] + ':', reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon() )


Directory  ./reconstructed-shapes/385Ma  already exists
Paleolocation (LAT, LON) for Shaffer-Rohling_Coreat 385Ma: (-35.651247730690336, -46.72879540520652)
Paleolocation (LAT, LON) for I35_Sycamore_Southat 385Ma: (-37.364561924648925, -47.04594472580862)
Paleolocation (LAT, LON) for Kansas_OK_Outcropat 385Ma: (-37.0865110097868, -43.709706588477836)
Paleolocation (LAT, LON) for Hamsten_Unit_Coreat 385Ma: (-34.56183582073328, -46.43152903148178)
Paleolocation (LAT, LON) for Bakkenat 385Ma: (-18.689849309372608, -42.62751961896904)
Directory  ./reconstructed-shapes/380Ma  already exists
Paleolocation (LAT, LON) for Shaffer-Rohling_Coreat 380Ma: (-33.947256222531635, -46.53381617456771)
Paleolocation (LAT, LON) for I35_Sycamore_Southat 380Ma: (-35.66265828098398, -46.826668663469015)
Paleolocation (LAT, LON) for Kansas_OK_Outcropat 380Ma: (-35.36459107205279, -43.565734502930944)
Paleolocation (LAT, LON) for Hamsten_Unit_Coreat 380Ma: (-32.855910288027204, -46.25309442480197)
Paleolocati

In [31]:
# # reconstruct polygons, i.e. area shapes
wedgesNames = ['catskill', 'price-pocono', 'pennington']

for wedgeCount, wedgeName in enumerate(wedgesNames):
    
    # load wedges coordinates
    df_wedge = pd.read_csv('./'+ wedgeName +'-wedges.csv',sep=',')
    
    for expCount, exp in enumerate(expts):

        # Create target directory & all intermediate directories if don't exists
        dirName = './reconstructed-shapes/'+ labels[expCount]
        if not os.path.exists(dirName):
            os.makedirs(dirName)
            print("Directory " , dirName ,  " Created ")
        else:    
            print("Directory " , dirName ,  " already exists") 

        # put the points into a feature collection, using Lat,Lon coordinates from dataframe
        polygon_points = []
        polygon_feature = []
        for index,row in df_wedge.iterrows():
            point = pygplates.PointOnSphere(float(row.LAT),float(row.LON))
            polygon_points.append(point)

        #add_polyline_feature_from_points(polyline_features, polyline_points, row)
        polygon = pygplates.PolygonOnSphere(polygon_points)
        polygon_feature = pygplates.Feature() # 'unclassified' feature
        polygon_feature.set_geometry(polygon)

        # The partition points function can then be used as before
        partitioned_polygon_feature = pygplates.partition_into_plates(static_polygons, rotation_model, polygon_feature) 

        # Reconstruct the shapes
        reconstructed_polyline_feature_geometries = []
        pygplates.reconstruct(partitioned_polygon_feature, rotation_model, dirName +'/' + wedgeName + '-wedge.shp', float(ages[expCount]) ) 


Directory  ./reconstructed-shapes/385Ma  already exists
Directory  ./reconstructed-shapes/380Ma  already exists
Directory  ./reconstructed-shapes/375Ma  already exists
Directory  ./reconstructed-shapes/370Ma  already exists
Directory  ./reconstructed-shapes/366Ma  already exists
Directory  ./reconstructed-shapes/359Ma  already exists
Directory  ./reconstructed-shapes/354Ma  already exists
Directory  ./reconstructed-shapes/349Ma  already exists
Directory  ./reconstructed-shapes/344Ma  already exists
Directory  ./reconstructed-shapes/339Ma  already exists
Directory  ./reconstructed-shapes/333Ma  already exists
Directory  ./reconstructed-shapes/327Ma  already exists
Directory  ./reconstructed-shapes/385Ma  already exists
Directory  ./reconstructed-shapes/380Ma  already exists
Directory  ./reconstructed-shapes/375Ma  already exists
Directory  ./reconstructed-shapes/370Ma  already exists
Directory  ./reconstructed-shapes/366Ma  already exists
Directory  ./reconstructed-shapes/359Ma  already